In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import json
from requests_html import HTMLSession

session = HTMLSession()

metrika_headers = {
    'GET': '/management/v1/counters HTTP/1.1',
    'Host': 'api-metrika.yandex.net',
    'Authorization': '',
    'Content-Type': 'application/x-yametrika+json'
    }

#n = "30"
#counter = "52015214"
counters = ['52015214', '40011245', '53322454']
report_period_days = 180
# счетчики: 52015214 - , 40011245 - , 53322454 - 

In [2]:
def get_metrika_templates(metrika_url,metrika_headers,counter,report_period_days):

    json_content_response = session.get(metrika_url, headers=metrika_headers).json()

    columns = []

    for i in json_content_response['query']['dimensions']:
        columns.append(i.replace("ym:s:",""))
    for i in json_content_response['query']['metrics']:
        columns.append(i.replace("ym:s:",""))

    datalists = dict()
    for item in columns:
        datalists[item] = []

    for i in json_content_response['data']:
        datalists[columns[0]].append(i['dimensions'][0]['name'])
        datalists[columns[1]].append(i['dimensions'][1]['name'])
        for j in range(len(json_content_response['data'][0]['metrics'])):
            datalists[columns[j+2]].append(i['metrics'][j])

    return pd.DataFrame(datalists)

In [3]:
for counter in counters:
    metrika_templates = {'traffic': 'https://api-metrika.yandex.net/stat/v1/data?dimensions=ym:s:datePeriodmonth,ym:s:counterID&metrics=ym:s:visits,ym:s:users,ym:s:pageviews,ym:s:percentNewVisitors,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds&date1={}daysAgo&date2=today&group={}&ids={}&limit=10000'.format(report_period_days, "month", counter), 'convertion': 'https://api-metrika.yandex.net/stat/v1/data?dimensions=ym:s:goal,ym:s:counterID&metrics=ym:s:visits,ym:s:users,ym:s:pageviews,ym:s:sumGoalReachesAny,ym:s:anyGoalConversionRate&date1={}daysAgo&date2=today&group={}&ids={}&limit=10000'.format(report_period_days,"month",counter), 'DirectPlatform': 'https://api-metrika.yandex.net/stat/v1/data?dimensions=ym:s:lastsignDirectPlatform,ym:s:counterID&metrics=ym:s:visits,ym:s:users,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds&date1={}daysAgo&date2=today&group={}&ids={}&limit=10000'.format(report_period_days,"month",counter), 'geography': 'https://api-metrika.yandex.net/stat/v1/data?dimensions=ym:s:regionCountry,ym:s:counterID&metrics=ym:s:visits,ym:s:users,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds&date1={}daysAgo&date2=today&group={}&ids={}&limit=10000'.format(report_period_days,"month",counter), 'age': 'https://api-metrika.yandex.net/stat/v1/data?dimensions=ym:s:ageInterval,ym:s:counterID&metrics=ym:s:visits,ym:s:users,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds&date1={}daysAgo&date2=today&group={}&ids={}&limit=10000'.format(report_period_days,"month",counter), 'gender': 'https://api-metrika.yandex.net/stat/v1/data?dimensions=ym:s:gender,ym:s:counterID&metrics=ym:s:visits,ym:s:users,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds&date1={}daysAgo&date2=today&group={}&ids={}&limit=10000'.format(report_period_days,"month",counter), 'interests': 'https://api-metrika.yandex.net/stat/v1/data?dimensions=ym:s:interest2d1,ym:s:counterID&metrics=ym:s:visits,ym:s:users,ym:s:affinityIndexInterests2,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds&date1={}daysAgo&date2=today&group={}&ids={}&limit=10000'.format(report_period_days,"month",counter)}
    for key, value in metrika_templates.items():
        #locals()['ya_data_{}'.format(key)] = get_metrika_templates(value, metrika_headers,counter,n)
        get_metrika_templates(value, metrika_headers,counter,report_period_days).to_csv('/Users/aliyadavletshina/Desktop/UD_Group/udg/general_data/gen_{}_{}.csv'.format(counter, key))

In [4]:
tables = []
for key, value in metrika_templates.items():
    tables.append(key)

In [5]:

for i in tables:
    result = pd.DataFrame()
    for j in counters:
        df = pd.read_csv("/Users/aliyadavletshina/Desktop/UD_Group/udg/general_data/gen_{}_{}.csv".format(j,i))
        os.remove('/Users/aliyadavletshina/Desktop/UD_Group/udg/general_data/gen_{}_{}.csv'.format(j,i))
        result = result.append(df, ignore_index=True)
    result.to_csv("/Users/aliyadavletshina/Desktop/UD_Group/udg/general_data/{}.csv".format(i))